# Demonstration
This is a demonstration of my work. This is neither the paper or the software. Simply a naive walk through of my program.

# 1. Generate time series data from a dynamic system

## Setting up libraries

In [1]:
from collections import defaultdict

import numpy as np

from tools.lorenz import Lorenz63
from tools.utils import iterate_solver, Runge_Kutta
%matplotlib widget


In [2]:
from pylab import *
from mpl_toolkits.mplot3d import Axes3D

## Here, I am using [Lorenz system](https://en.wikipedia.org/wiki/Lorenz_system) with parameter `s` = 10, `r` = 28, and  `b` = 8/3. This set of parameter gives [all the solution bounded](https://www.math.toronto.edu/kzhang/teaching/courses/mat332-2022/_8-lorenz-system/) (do not escape to infinity). With initial condition: -1, 1, 18.4

In [3]:
x, t = iterate_solver(Runge_Kutta, Lorenz63, [-1., 1., 18.4], 0, 0.01, 50.)
figure("Numerical Solution from 0 to 50")
title("Lorenz Attractor Simulation")
plot(t, x[:, 1], 'b-')

xlabel("Time")  # Label for the X-axis
ylabel("Variable State")
show()
# matplotlib.pyplot.close()

In [4]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
ax.plot(x[:, 0], x[:, 1], x[:, 2])

ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')

plt.show()

In [5]:
type(x)

In [6]:
len(x)

In [7]:
import copy
x_gapped = copy.deepcopy(x)

In [8]:
x_gapped[2000:2100] = np.nan
len(x_gapped), len(t)

In [9]:
nan_count = np.sum(np.isnan(x_gapped)) / 3
nan_count

In [10]:
fig = plt.figure()
plot(t, x_gapped[:, 0], 'b-')
plot(t, x_gapped[:, 1], 'r-')
plot(t, x_gapped[:, 2], 'g-')

ts = x_gapped[:, 0]
plt.xlim(min(t), max(t))  # Set x limits to cover the full range of t
plt.show()

In [11]:
len(x_gapped[:, 0]), len(t)

In [12]:
print(t.shape)
print(x_gapped.shape)
print(x_gapped[:, 0].shape)

In [13]:
fig = plt.figure()
plot(t, ts, 'g-')
# matplotlib.pyplot.close()

In [14]:
len(ts)

In [15]:
indices = np.arange(0, len(ts), 10)

In [16]:
indices

In [17]:
v_1_s = ts[indices]

In [18]:
indices2 = (indices + 1)
indices2 = indices2[:-1]
v_2_s = ts[indices2]

In [19]:
# indices2

In [20]:
indices3 = (indices + 2)
indices3 = indices3[:-1]
v_3_s = ts[indices3]

In [21]:
v_1_s = v_1_s[:-1]

In [22]:
vectors = column_stack((v_1_s, v_2_s, v_3_s))

In [23]:
np.sum(np.isnan(vectors))

In [24]:
def get_dis_matrix(vectors):
    dis_matrix = np.zeros((len(vectors), len(vectors)))
    for m, i in enumerate(vectors):
        for n, j in enumerate(vectors):
            if any(isnan(i)) or any(isnan(j)):
                dis_matrix[m, n] = np.inf
                continue
            if m == n:
                dis_matrix[m, n] = np.inf
                continue
            dis_matrix[m, n] = linalg.norm(i - j)
    return dis_matrix
                

In [25]:
linalg.norm(vectors[0] - vectors[1])

In [26]:
vectors

In [27]:
dis_matrix = get_dis_matrix(vectors)

In [28]:
dis_matrix

In [29]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
ax.plot(vectors[:, 0], vectors[:, 1], vectors[:, 2])

ax.set_xlabel('X Coordinate')
ax.set_ylabel('Y Coordinate')
ax.set_zlabel('Z Coordinate')

plt.show()

In [30]:
# !pip install ipympl

In [31]:
fig = plt.figure()
plot(arange(len(vectors)), vectors[:, 0], 'g-')
plot(arange(len(vectors)), vectors[:, 1], 'r-')
plot(arange(len(vectors)), vectors[:, 2], 'b-')

In [32]:
def get_breaking_points(vectors):
    # Create a boolean mask where each sub-array is checked for NaN
    contains_nan = np.array([np.any(np.isnan(vector)) for vector in vectors])
    
    # Find the index of the first sub-array containing NaN
    first_nan_index = np.argmax(contains_nan)
    # first_with_nan = arrays[first_nan_index] if np.any(contains_nan) else None
    
    # Find the index of the last sub-array containing NaN
    # np.max(np.where(contains_nan)[0]) provides the last index with True
    last_nan_index = np.max(np.where(contains_nan)[0]) if np.any(contains_nan) else None
    # last_with_nan = arrays[last_nan_index] if last_nan_index is not None else None
    
    return first_nan_index, last_nan_index

In [33]:
first_nan_index, last_nan_index = get_breaking_points(vectors)

In [34]:
first_nan_index

In [35]:
last_nan_index

In [36]:
vectors[first_nan_index - 1]

In [37]:
vectors[last_nan_index]

In [38]:
last_valid_v_index = first_nan_index - 1
next_valid_v_index = last_nan_index + 1
l = next_valid_v_index - last_valid_v_index

In [39]:
dis_matrix[last_valid_v_index]

In [40]:
closest_point_index = np.argmin(dis_matrix[last_valid_v_index])

In [41]:
closest_point_index

In [42]:
dis_matrix[last_valid_v_index, closest_point_index]

In [43]:
vectors[closest_point_index], vectors[first_nan_index - 1], first_nan_index - 1, closest_point_index

In [44]:
# def get_closest_point_index()
def get_one_branch(index: int, vectors: np.array, rest_steps: int, forward_branches_df: dict ,forward_branches_df_reverse,  forward: bool = True):
    # root_node = vectors[index]
    # forward_branches_df[index]
    # rest steps = l - j 
    
    #        next_node
    #         /
    # vectors[index] 
    #         \
    #        next_node_2
    
    step = 1 if forward else -1
    
    curr_root = index
    curr_child = curr_root + step
    print("start of the branch -------")
    for i in range(rest_steps):
        if curr_child >= len(vectors):
            # print(f"curr_child: {curr_child}")
            break
        if np.any(isnan(vectors[curr_child])):
            print(f"curr_child is NaN")
            break
        forward_branches_df[curr_root].add(curr_child)
        forward_branches_df_reverse[curr_child] = curr_root
        # print(f"check vectors[curr_child]: {vectors[curr_child]}")
        # forward_branches_bf[curr_layer_num + i].append(curr_root)
        curr_root = curr_root + step
        curr_child = curr_root + step
    
    print(f"finished current branch -------- ")
    
    return forward_branches_df, forward_branches_df_reverse

def get_branches_forward(last_valid_v_index: int, next_valid_v_index: int, n_f: int, r: int, vectors: np.array, dis_matrix: np.array):
    # TODO: finish the second condition: or until a valid seccussor
    
    l = next_valid_v_index - last_valid_v_index - 1
    # layer_num = (l // r) + 1 # +1 to make give some extra space
    
    forward_branches_df = defaultdict(set)
    forward_branches_df_reverse = dict()
    # forward_branches_bf = [[] for _ in range(layer_num)]
    
    # get_one_branch(index = last_valid_v_index, vectors=vectors, rest_steps=l - last_valid_v_index, forward_branches_df=forward_branches_df, forward_branches_bf=forward_branches_bf, curr_layer_num=0)
    
    curr_node_index = last_valid_v_index 
    
    for i in range(n_f): # + 1 because zero included
        if last_valid_v_index + (i * (1 + r)) >= next_valid_v_index:
            break
        print("test")
        # jump_to_node_index = last_valid_v_index + ((i + 1) * r)
        jump_to_index = np.argmin(dis_matrix[curr_node_index])
        print(f"step: {i}")
        print(f"jump_to_index: {jump_to_index}")
        forward_branches_df[curr_node_index].add(jump_to_index)
        forward_branches_df_reverse[jump_to_index] = curr_node_index
        forward_branches_df, forward_branches_df_reverse = get_one_branch(index = jump_to_index, vectors=vectors, rest_steps= l - (i * (r + 1)) - 1, forward_branches_df=forward_branches_df, forward = True, forward_branches_df_reverse = forward_branches_df_reverse)
        curr_node_index = jump_to_index + r
    return forward_branches_df, forward_branches_df_reverse

def get_branches_backward(last_valid_v_index: int, next_valid_v_index: int, n_b: int, r: int, vectors: np.array, dis_matrix: np.array):
    # TODO: finish the second condition: or until a valid seccussor
    
    l = next_valid_v_index - last_valid_v_index - 1
    # layer_num = (l // r) + 1 # +1 to make give some extra space
    
    back_branches_df = defaultdict(set)
    back_branches_df_reverse = dict()
    # forward_branches_bf = [[] for _ in range(layer_num)]
    
    # get_one_branch(index = last_valid_v_index, vectors=vectors, rest_steps=l - last_valid_v_index, forward_branches_df=forward_branches_df, forward_branches_bf=forward_branches_bf, curr_layer_num=0)
    
    curr_node_index = next_valid_v_index 
    
    for i in range(n_b): # + 1 because zero included
        if last_valid_v_index + (i * (1 + r)) >= next_valid_v_index:
            break
        print("test")
        # jump_to_node_index = last_valid_v_index + ((i + 1) * r)
        jump_to_index = np.argmin(dis_matrix[curr_node_index])
        # print(f"step: {i}")
        # print(f"jump_to_index: {jump_to_index}")
        back_branches_df[curr_node_index].add(jump_to_index)
        back_branches_df_reverse[jump_to_index] = curr_node_index
        back_branches_df, back_branches_df_reverse = get_one_branch(index = jump_to_index, vectors=vectors, rest_steps= l - (i * (r + 1)) - 1, forward_branches_df=back_branches_df, forward_branches_df_reverse=back_branches_df_reverse,  forward = False)
        curr_node_index = jump_to_index - r
    return back_branches_df, back_branches_df_reverse

In [45]:
forward_branches_df, forward_branches_df_reverse = get_branches_forward(last_valid_v_index=last_valid_v_index, 
                                   next_valid_v_index=next_valid_v_index, 
                                   n_f=5, r=2, vectors=vectors, dis_matrix=dis_matrix)

In [46]:
vectors[closest_point_index], vectors[first_nan_index - 1], last_valid_v_index, next_valid_v_index, next_valid_v_index - last_valid_v_index

In [47]:
forward_branches_df_reverse

In [48]:
forward_branches_df

In [49]:
backward_branches_df, back_branches_df_reverse = get_branches_backward(last_valid_v_index=last_valid_v_index, 
                                   next_valid_v_index=next_valid_v_index, 
                                   n_b=5, r=2, vectors=vectors, dis_matrix=dis_matrix)

In [50]:
back_branches_df_reverse

In [51]:
backward_branches_df

In [52]:
def tree_to_layers(tree, queue):
    layers = []
    # queue = [("root", 0)]  # Queue of tuples (node, layer_index)

    while queue:
        current_node, layer = queue.pop(0)
        
        # Ensure the layer exists in the layers list
        if len(layers) <= layer:
            layers.append([])
        
        # Append the current node to its respective layer
        layers[layer].append(current_node)
        
        # Enqueue all children of the current node
        for child in tree.get(current_node, []):
            queue.append((child, layer + 1))
    
    return layers

# Example tree
tree = {
    "root": ["child1", "child2"],
    "child1": ["grandchild1", "grandchild2", "grandchild3"],
    "grandchild2": ["a", 'b', "c"],
    "child2": ["child2's child1", "child2's child3"],
}

# Convert tree to layers
# layers = tree_to_layers(tree)
# print(layers)


In [53]:
forward_layer = tree_to_layers(forward_branches_df, queue = [(199, 0)])

In [54]:
forward_layer

In [55]:
len(forward_layer)

In [56]:
backward_layer = tree_to_layers(backward_branches_df, queue = [(210, 0)])

In [57]:
backward_layer

In [58]:
len(backward_layer)

In [59]:
def get_closest_points_layer(vectors, forward_layer, backward_layer):
    l = len(forward_layer) - 1
    closest_dis = float('inf')
    closest_forward_index = 0
    closest_forward_index_sub = 0
    closest_backward_index_sub = 0
    for i in range(l):
        forward_vectors = forward_layer[i]
        backward_vectors = backward_layer[l - i]
        forward_sub_index, backward_sub_index, dis = get_closest_points_one_layer(vectors, forward_vectors, backward_vectors)
        if dis < closest_dis:
            closest_dis = dis
            closest_forward_index = i
            closest_forward_index_sub = forward_sub_index
            closest_backward_index_sub = backward_sub_index
    return closest_dis, closest_forward_index, closest_forward_index_sub, closest_backward_index_sub

def get_closest_points_one_layer(vectors, forward_vectors, backward_vectors):
    min_dis = float('inf')
    forward_sub_index = 0
    backward_sub_index = 0
    for n, i in enumerate(forward_vectors):
        for m, j in enumerate(backward_vectors):
            curr_dis = linalg.norm(vectors[i] - vectors[j])
            if curr_dis < min_dis:
                min_dis = curr_dis
                forward_sub_index = n
                backward_sub_index = m
    return forward_sub_index, backward_sub_index, min_dis

In [60]:
closest_dis, closest_forward_index, closest_forward_index_sub, closest_backward_index_sub = get_closest_points_layer(vectors=vectors, forward_layer=forward_layer, backward_layer=backward_layer)
closest_dis, closest_forward_index, closest_forward_index_sub, closest_backward_index_sub

In [61]:
linalg.norm(vectors[387] - vectors[187])

In [62]:
closest_forward_index

In [63]:
def get_gap_vector_index_list(closest_forward_index, closest_forward_index_sub, closest_backward_index_sub, forward_branches_df_reverse, backward_branches_df_reverse, forward_layer, backward_layer, l:int):
    print(f"closest_forward_index: {closest_forward_index}")
    forward_vector_index = forward_layer[closest_forward_index][closest_forward_index_sub]
    backward_vector_index = backward_layer[l - closest_forward_index][closest_backward_index_sub]
    vector_index_list = list()
    curr_backward_vector_index = backward_vector_index
    curr_forward_vector_index = forward_vector_index
    for i in range(closest_forward_index):
        vector_index_list.insert(0, curr_forward_vector_index)
        curr_forward_vector_index = forward_branches_df_reverse[curr_forward_vector_index]
    
    for i in range(l - closest_forward_index):
        vector_index_list.append(curr_backward_vector_index)
        curr_backward_vector_index = backward_branches_df_reverse[curr_backward_vector_index]
    
    return vector_index_list

In [64]:
gap_vector_index_list = get_gap_vector_index_list(closest_forward_index=closest_forward_index, closest_forward_index_sub=closest_forward_index_sub, closest_backward_index_sub=closest_backward_index_sub, forward_layer=forward_layer, backward_layer=backward_layer, forward_branches_df_reverse=forward_branches_df_reverse, backward_branches_df_reverse=back_branches_df_reverse, l = next_valid_v_index - last_valid_v_index - 1)

In [65]:
gap_vector_index_list

In [66]:
len(gap_vector_index_list)

In [75]:
vector_list = [vectors[i] for i in gap_vector_index_list]

In [76]:
vector_list = row_stack(vector_list)
vector_list

In [77]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# 
# # Scatter plot
# ax.plot(vector_list[:, 0], vector_list[:, 1], vector_list[:, 2])
# 
# # Scatter plot
# 
# ax.set_xlabel('X Coordinate')
# ax.set_ylabel('Y Coordinate')
# ax.set_zlabel('Z Coordinate')
# 
# plt.show()
# matplotlib.pyplot.close()

In [78]:
# vector_sample = vectors[190:220]
# ax.plot(vector_sample[:, 0], vector_sample[:, 1], vector_sample[:, 2])

In [79]:
# fig = plt.figure()

# vector_sample = vectors[190:220]
# ax = fig.add_subplot(111, projection='3d')
# 
# # Scatter plot
# ax.plot(vector_sample[:, 0], vector_sample[:, 1], vector_sample[:, 2])
# 
# ax.set_xlabel('X Coordinate')
# ax.set_ylabel('Y Coordinate')
# ax.set_zlabel('Z Coordinate')
# 
# plt.show()
# matplotlib.pyplot.close()

In [160]:
vectors[210]

In [162]:
fig = plt.figure()
plot(arange(len(vectors)), vectors[:, 0])
# plot(t, vector_list[:, 0])

In [163]:
vector_draw = np.vstack([vectors[199], vector_list, vectors[210]])
plot(arange(199, 211), vector_draw[:, 0])

In [82]:
vectors[209]

In [83]:
len(vector_list)

In [113]:
def get_distance_list(vector, vectors):
    dis_list = np.zeros(len(vectors))
    for n, j in enumerate(vectors):
        if any(isnan(j)):
            dis_list[n] = np.inf
            continue
        dis_list[n] = linalg.norm(vector - j)
    return dis_list

def F_j_minus_one_half(j: int, vector_list, vectors, t):
    # vector_list is the gapped vector
    # closes to x_j
    # second close to x_j
    # predecesor of both
    x_j = vector_list[j]
    dis_list = get_distance_list(vector=x_j, vectors=vectors)
    closest_x_j_index = np.argmin(dis_list)
    while closest_x_j_index - 1 < 0 or any(isnan(vectors[closest_x_j_index - 1])):
        dis_list[closest_x_j_index] = np.inf
        closest_x_j_index = np.argmin(dis_list)
    x_j_bar = vectors[closest_x_j_index]
    p_x_j_bar = vectors[closest_x_j_index - 1]
    
    dis_list[closest_x_j_index] = np.inf
    sec_closest_x_j_index = np.argmin(dis_list)
    
    while sec_closest_x_j_index - 1 < 0 or any(isnan(vectors[sec_closest_x_j_index - 1])):
        dis_list[sec_closest_x_j_index] = np.inf
        sec_closest_x_j_index = np.argmin(dis_list)
    
    x_j_bar_bar = vectors[sec_closest_x_j_index]
    p_x_j_bar_bar = vectors[sec_closest_x_j_index - 1]
    delta_t = t[1] - t[0]
    
    # print(f"x_j_bar: {x_j_bar}, p_x_j_bar: {p_x_j_bar}, delta_t: {delta_t}, x_j_bar_bar: {x_j_bar_bar}, p_bar_bar_bar: {p_x_j_bar_bar}")
    
    return (x_j_bar - p_x_j_bar + x_j_bar_bar - p_x_j_bar_bar) / (2 * delta_t)

F_j_minus_one_half(1, vector_list[:, 0], vectors, t)

In [114]:
dis_list = zeros_like(len(vectors))

In [142]:
def J1(vector_list, vectors, t):
    w = vector_list
    l = len(w)
    sum_squares = 0
    for j in range(l):
        delta_w = (w[j] - w[j-1]) / t[j + 1] - t[j]
        # print(f"w[j]: {w[j]}")
        # print(f"delta_w: {delta_w}")
        F_value = F_j_minus_one_half(j=j, vectors=vectors, vector_list=vector_list, t=t)  # Assuming F is already defined
        # print(f"F_value: {F_value}")
        # sum_squares += (delta_w - F_value)**2
        sum_squares += linalg.norm(delta_w - F_value) ** 2
    return sum_squares

def objective_function_to_minimize(v_f):
    vector_list = v_f.reshape((10, 3))
    return J1(vector_list, vectors, t)

In [144]:
from scipy.optimize import minimize

result = minimize(objective_function_to_minimize, vector_list.flatten(), method='L-BFGS-B')

In [143]:
objective_function_to_minimize(vector_list)

In [146]:
result.x

In [147]:
vector_list

In [148]:
result.fun

In [151]:
result_array = result.x.reshape((10, 3))

In [154]:
fig = figure()
plot(arange(len(vector_list)), vector_list[:, 0])
plot(arange(len(vector_list)), result_array[:, 0])

In [167]:
fig = plt.figure()
plot(arange(len(vectors)), vectors[:, 0])
result_draw = np.vstack([vectors[199], result_array, vectors[210]])
plot(arange(199, 211), result_draw[:, 0])